## 1 Baseline (Nuhn, 2013)

Basically the baseline is a Breadth First Search (BFS) algorithm with pruning.

### 1.1 Notation:

We use the same notation by Nuhn et al in their 2013 paper<sup>[1]</sup>.

The ciphertext is: $f_1^N=f_1,…,_fi,…,f_N,\ f_i ∈ V_f$

The plaintext is: $e_1^N=e_1,…, e_i,…,e_N,\ e_i ∈ V_e$

Decipher function: $\phi :V_f → V_e$

We need to find the $\phi​$ such that it maximize $Pr(\phi(f_1),…,\phi(f_N))​$.

### 1.2 Algorithm: 

Since the baseline algorithm is well explained in homework requirement so we do not repeat it in the notebook. 

But we will explain how we choose the `score` function and `ext_order` in detail.

![Figure 1](http://anoopsarkar.github.io/nlp-class/assets/img/decipherbeam.png)

### 1.3 Extension order we use in baseline:

We sort the symbols in ciphertext based on their frequency in descending order in a python list. Then we will do beam search for cipher symbols in this order.

We have improved this extension order algorithm. See the "improvements" section in our nodebook.

### 1.4 Score function we use:

Since we will get potential plaintext sequences in the middle of deciphering which are discrete, we need to score the discrete sequences.

 We divided the sequences into three categories.

1. The sequence contains start symbol (contains the first letter in the whole text)
2. The sequence contains end symbol (contains the last letter in the whole text)
3. The sequences not contain either start or end symbols.

Suppose we have a plaintext sequence $e_ie_{i+1}....e_j$, the start & end symbol are $\$$ .

In first case, we compute the score as $\Pi_k Pr(e_k|\$e_ie_{i+1}...e_{k-1})$

In second case, we compute the score as $(\Pi_k Pr(e_k|e_ie_{i+1}...e_{k-1}))*Pr(\$|e_i...e_j)$

In third case, we compute the score as $(\Pi_k Pr(e_k|e_ie_{i+1}...e_{k-1}))$, ignoring start or end symbols.

(We actually compute the log likelihood in our program.)

We tried some improvements of this score function, please see the "improvements" sections. 

### 1.5 Running Time analysis:

Suppose the number of symbols of ciphertext is $N_f$. The number of symbols of plaintext is $N_e$. We don't consider `ext_limits` in this analysis.

- Without pruning:
  - We need to search every possible mapping from cipher to plaintext.
  - For each letter in $V_f$ it could map to any letter in $V_e$
  - So the possible number of mappings is $N_e^{N_f}$
- With pruning with $beamsize = B$:
  - In each iteration we only keep at most $B$ mappings.
  - So the search tree has at most $B$ leaf nodes.
  - So the complexity is $N_f*B*N_e$, which is much smaller than the one without pruning.

## 2 Improvements

### 2.1 Ext order (Nuhn, 2014)

The `EXT_ORDER` (Extension Order) decides the order in which the symbols are searched and scored. Generating the `EXT_ORDER` means deciding the order of symbols to be searched. Our method performs beam search over the possible extension order sets that similar to the deciphering process. The beam search starts from empty symbol set, and expands the beam every time a new symbol goes in. To the beam search, the generated orders are scored by the function
``` python
def contiguous_score(cipher, order):
    order = set(order)
    count = 0
    ngrams = defaultdict(int)
    for c in cipher:
        if c in order:
            count += 1
            ngrams[min(lm_order, count)] += 1
        else:
            count = 0

    score = 0
    for k, v in ngrams.items():
        score += contiguous_score_weights[k] * v
    return score
```
This function gives higher scores to orders that put more evenly distributed characters at first. In other words, it scores more to more informative orders. Actually, we think that the performance can be further improved by using better function to score the orders.

### 2.2 Beamsize decay

In experiments we found that the correct result are likely to be pruned in early stage and in later stage where more ciphers are deciphered correct result usually gets a better score. So in order to better utilize computing resource and improve efficency we implemented beamsize decay. For the first half of the ciphers, the beamsize remain unchanged. For the second half, the beamsize are decayed exponentially by a factor of 0.85.

``` python
def dynamic_beamsize(cipher, beamsize):
    num_symbols = len(set(cipher))
    beamsizes = [beamsize] * (num_symbols)
    for i in range(num_symbols // 2, num_symbols):
        beamsizes[i] = int(beamsize * (0.85 ** (i - num_symbols//2)))
    return beamsizes

```

### 2.3 Multiprocessing

In order to improve efficency, we implemented multiprocessing. When scoring the partially deciphered string, instead of scoring it as a whole using `score_bitstring`, we split it into non contiguous substrings and use multiple process to score each of them. We use different scoring function for begin, end, and middle substrings to include to probability of start and end.

### 2.4 Dynamic ext_limits

Based on the true mappings of 408 zodiac cipher, we give the english letters limits of 4 except the limit of ‘e’ is 7. We do it because we don’t want it to be too specific about the zodiac cipher but also somewhat be helpful, since 7 is a pretty large limit for e and 4 for the rest is large usually enough. We hoped this could somehow be helpful but not sure if it really makes a big difference.

## 3 Attempts

### 3.1 Predict unknown letters and adopt frequency matching (Kambhatla, 2018)

This attempt is based on the beam search method with the ext order improvement. Say we are extending mapping $Φ$ to mapping $Φ'$ with $(f, e)$ (ciphertext symbol $f$ is mapped to plaintext symbol $e$), and we are scoring the new mapping $Φ'$.

Before this attempt, $SCORE_{Φ'} = lm.score\_bitstring(S_{Φ'})$. The score functions of lm return a negative number. The closer it is to 0, i.e. the bigger it is, the better $S$/$Φ'$ is.

#### 3.1.1 Implementation

##### 3.1.1.1 Predict unknown letters with neural language models

We use neural language models to replace unknown letters (ciphertext letters not yet in $Φ'$). Say $S_{Φ'}$ = "ab\_ba\_\_a\_" , we want to replace '\_' with the most possible plaintext letters, $e_i$. With the neural language model given to us, $e_i = next\_chars(S_{Φ'}[:i-1], ...)$. However, the next_chars() function does not always return the most likely successors. We will turn to ngram for help in such cases. We choose the letter $e_i$ that maximizes $lm.score(S_{Φ'}[:i])$.

With a complete sentence predicted, for each $f_i$ (f in the i-th position in the ciphertext), $SCORE_{Φ'} += nlm.score\_next(S_{Φ'}[:i-1], e_i)$. (The initial value of $SCORE_{Φ'}$ is $SCORE_Φ.$)

Note that the score functions of nlm return a positive number. The smaller it is, the better $S$/$Φ'$ is.

##### 3.1.1.2 Update score using frequency matching

Let $freq(f)$ denote the frequency of letter f in ciphertext and $freq(e)$ denote the frequency of letter e in plaintext. Suppose $Φ' = Φ ∪ (f,e)$. $FREQ\_MATCHING\_SCORE_{Φ'} = |log(freq(f)/freq(e))|$. The smaller the score it is, the closer $freq(f)$ and $freq(e)$ is, and the better $(f,e)$ is. Therefore, $SCORE_{Φ'} += FREQ\_MATCHING\_SCORE_{Φ'}$

#### 3.1.2 Result

We finished the implementation, but the outcome is not that satisfying. We tested this with a simple cipher text, the result is hardly even English. When running with the Zodiac-408 ciphertext, it was so slow that we failed to see the final result.

#### 3.1.3 Reflection

The frequency matching score should help a lot for 1:1 substitution ciphers and when the cipher text is long enough. However, we doubt its effect on substitution substitution ciphers.

Also, we think that the prediction should happen when a certain percentage of letters have been deciphered. Say $S_{Φ'}$ = "a\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_a", it doesn't make much sense and it is a waste of time to predict all those letters between the a's. In those cases, lm.score\_bitstring() can be applied.

### 3.2 Ignoring unigrams and bigrams

At first we score every substring using the 6-gram language model. We examined in each iteration whether the correct mappings is pruned or not by printing out the worst score in the beams and the score for correct mappings(only a subset of ciphers that is already searched). If the correct score is better than the worst score, it means the correct mappings is still in the beams. And we found that in the early stage where few ciphers are deciphered(where the partially deciphered text is full of unigrams and bigrams), the score for the correct mappings is very bad, which makes it be pruned very early. And later searching leads to very bad result because it starts off based on very wrong mappings. 

So we tried to ignore uingrams and bigrams when scoring. We tried to ignore unigrams only and the correct result are less easier to be pruned but still not good enough. If ignoring both unigrams and bigrams, we need to make the beamsize sufficiently large at the start because the scores are all 0 for the first few ciphers. 

We are still running the program. 

## Reference

[1] Malte Nuhn, Julian Schamper, and Hermann Ney. 2013. Beam search for solving substitution ciphers. In Proceedings of the 51st Annual Meeting of the Association for Computational Linguistics (Volume 1: Long Papers), volume 1, pages 1568–1576.

[2] Malte Nuhn, Julian Schamper, and Hermann Ney. 2014. Improved decipherment of homophonic ciphers. In Proceedings of the 2014 Conference on Empirical Methods in Natural Language Processing (EMNLP), pages 1764–1768.

[3] Nishant Kambhatla, Anahita Mansouri Bigvand, and Anoop Sarkar. 2018. Decipherment of substitution ciphers with neural language models.
